In [ ]:
!pip freeze | grep scikit-learn

In [ ]:
import pickle
import pandas as pd

In [ ]:
with open('model.bin', 'rb') as f_in:
    dv, lr = pickle.load(f_in)

In [ ]:
categorical = ['PUlocationID', 'DOlocationID']

def read_data(filename):
    df = pd.read_parquet(filename)
    
    df['duration'] = df.dropOff_datetime - df.pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    
    return df

In [ ]:
df = read_data('https://nyc-tlc.s3.amazonaws.com/trip+data/fhv_tripdata_2021-02.parquet')

In [ ]:
dicts = df[categorical].to_dict(orient='records')
X_val = dv.transform(dicts)
y_pred = lr.predict(X_val)

In [ ]:
import numpy as np

np.mean(y_pred)

In [ ]:
year=2021
month=2


df['ride_id'] = f'{year:04d}/{month:02d}_' + df.index.astype('str')

In [ ]:
output_file = "output.parquet"
df.to_parquet(
    output_file,
    engine='pyarrow',
    compression=None,
    index=False
)